### packages 

In [1]:
import pandas as pd
import numpy as np
import logging, sys
logging.basicConfig(level=logging.INFO)

In [13]:
import sqlite3

def data_extraction():
    
    
    logging.info("import the sqlite data...")
    con = sqlite3.connect("C:\\Users\\Downloads\\bat.sqlite3") 
    cur = con.cursor()                                             

    device = []
    readings=[]
    for row in cur.execute('SELECT * FROM device;'):
        device.append(row)                                
    for row in cur.execute("SELECT * FROM readings"):
        readings.append(row)                             
    
    
    logging.info("complete...")
    
    logging.info("creating dataframe")
    device=pd.DataFrame(device)                           
    readings=pd.DataFrame(readings)

    device.rename(columns={0:"device_id",1:"parent_id",
                           2:"name",3:"device_type"},inplace=True)   #renaming features of device
    device
    readings.rename(columns={0:"device_id",1:"voltage",
                             2:"temperature"},inplace=True)          #renaming features of readings
    readings
    
    merge_data = (device.merge(readings, left_on='device_id', right_on='device_id') 
       .reindex(columns=['device_id', 'parent_id', 'name', 
                         'device_type','voltage','temperature'])) #combining both device and reasings data using common feature device_id
    
    
    
    #list of total modules for two stacks i.e.module 1 to 4 fore each stack(stack1 and 2)
    module_list=["module1","module2","module3","module4",
                 "module1","module2","module3","module4"] 
    #print(module_list)
    
    stack_list=["stack1","stack2"]                                #list of total stacks
    
    module_list=np.repeat(module_list,8)                          #modules for 8 cells 
    Modulelist=pd.DataFrame(module_list)                          #dataframe of total modules for 64 cells
    Modulelist.rename(columns={0:"module"},inplace=True)          
    
    stack_list=np.repeat(stack_list,32)                           #total stacks for 64 cells/8 modules
    stack_list=pd.DataFrame(stack_list)                           #Convert stacklist to dataframe 
    stack_list.rename(columns={0:"stack"},inplace=True)           
    
    MergeData=pd.concat([merge_data,Modulelist,stack_list],axis=1)#combined data of merge_data, module and stack dataframe
    logging.info("Dataframe created")
    
    
    
    
    return MergeData                                            
data_extraction()                                      

INFO:root:import the sqlite data...
INFO:root:complete...
INFO:root:creating dataframe
INFO:root:Dataframe created


,device_id,parent_id,name,device_type,voltage,temperature,module,stack
0,12,4.0,Cell 1,cell,2.599,32.1,module1,stack1
1,13,4.0,Cell 2,cell,2.846,32.5,module1,stack1
2,14,4.0,Cell 3,cell,2.931,40.1,module1,stack1
3,15,4.0,Cell 4,cell,3.379,36.7,module1,stack1
4,16,4.0,Cell 5,cell,0.087,34.4,module1,stack1
...,...,...,...,...,...,...,...,...
59,71,11.0,Cell 4,cell,3.182,34.0,module4,stack2
60,72,11.0,Cell 5,cell,3.235,33.5,module4,stack2
61,73,11.0,Cell 6,cell,3.358,36.0,module4,stack2
62,74,11.0,Cell 7,cell,2.797,39.1,module4,stack2


In [14]:
MergeData.to_csv("C:\\Users\\Downloads\\MergeData.csv")    #saving combined dataframe to csv file 

### cell_with_highest_temperature()
This function should return the device_id of the cell with the highest temperature.

In [ ]:
### def cell_with_highest_temperature(MergeData):                 # function1
    MergeData.sort_values(["temperature"],ascending=False)
    DeviceID=list(MergeData.device_id)
    Temp=list(MergeData.temperature)
    device_temp = dict(zip(DeviceID,Temp))
    device_id_of_the_cell=max(device_temp,key=device_temp.get)
    
    logging.info("device_id of the cell with highest temperature")
    logging.info(device_id_of_the_cell)
    return device_id_of_the_cell
cell_with_highest_temperature(MergeData)    

In [22]:
device_id_of_the_cell=cell_with_highest_temperature(MergeData)    

INFO:root:device_id of the cell with highest temperature
INFO:root:36


### module_with_highest_temperature()
This function should return the device_id of the module which has the cell with the highest temperature 


In [23]:
def module_with_highest_temperature(MergeData,device_id_of_the_cell):                # function2 
    module_of_the_cell=MergeData.module[MergeData.device_id==device_id_of_the_cell]
    
    logging.info("module of the cell with highest temperature")
    logging.info(module_of_the_cell)
    return module_of_the_cell

module_with_highest_temperature(MergeData,device_id_of_the_cell)
    

INFO:root:module of the cell with highest temperature
INFO:root:24    module4
Name: module, dtype: object


24    module4
Name: module, dtype: object

### averate_voltage_in module(device_id)
This function should return the average voltage of all the cells in all the modules in the stack represented by the device_id, Cells with Voltage below 0.1V are dead cells and should be excluded from this analysis.


In [24]:
def averate_voltage_in_module(MergeData):                       #fuction 3
    voltage_required=MergeData.voltage[MergeData.voltage>=0.1]
    voltage_required=voltage_required.mean()
    
    logging.info("average voltage of the cells excluding values equal and less than 0.1")
    logging.info(voltage_required)
    return voltage_required

averate_voltage_in_module(MergeData)
    

INFO:root:average voltage of the cells excluding values equal and less than 0.1
INFO:root:2.9813508771929826


2.9813508771929826